# Music Lyrics Through the Years

#### Import (possibly) necessary libraries

In [1]:
import pandas as pd
import numpy as np

import re
from unicodedata import normalize

import requests as rq
from bs4 import BeautifulSoup as bs
from selenium import webdriver

import pickle

from functions.webscraping import *

%load_ext autoreload
%autoreload 2

## Scrape song title and artist

In [2]:
test_url = 'https://top40weekly.com/1960-all-charts/'

In [3]:
top10s_1960 = yearly_top10s(test_url)
len(top10s_1960)

103

In [4]:
top10s_1960

[('NORTH TO ALASKA', 'Johnny Horton'),
 ('STAIRWAY TO HEAVEN', 'Neil Sedaka'),
 ('WONDERLAND BY NIGHT', 'Bert Kaempfert'),
 ('WHAT IN THE WORLD’S COME OVER YOU', 'Jack Scott'),
 ('EL PASO', 'Marty Robbins'),
 ('SIXTEEN REASONS', 'Connie Stevens'),
 ('ITSY BITSY TEENIE WEENIE YELLOW POLKA DOT BIKINI', 'Brian Hyland'),
 ('WALKING TO NEW ORLEANS', 'Fats Domino'),
 ('THE THEME FROM “A SUMMER PLACE”', 'Percy Faith and His Orchestra'),
 ('ALONE AT LAST', 'Jackie Wilson'),
 ('GO, JIMMY, GO', 'Jimmy Clanton'),
 ('THEME FROM “THE APARTMENT”', 'Ferrante and Teicher'),
 ('THE TWIST', 'Chubby Checker'),
 ('LET’S GO, LET’S GO, LET’S GO', 'Hank Ballard and the Midnighters'),
 ('DEVIL OR ANGEL', 'Bobby Vee'),
 ('WHEN WILL I BE LOVED', 'The Everly Brothers'),
 ('MY HEART HAS A MIND OF ITS OWN', 'Connie Francis'),
 ('I’M SORRY', 'Brenda Lee'),
 ('I WANT TO BE WANTED', 'Brenda Lee'),
 ('LET THE LITTLE GIRL DANCE', 'Billy Bland'),
 ('THAT’S ALL YOU GOTTA DO', 'Brenda Lee'),
 ('FOOTSTEPS', 'Steve Lawrence